<a href="https://colab.research.google.com/github/lake198/Study-AI/blob/main/%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92Day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[Section1] 強化学習

強化学習は機械学習の一種である。教師有り・教師なし学習が与えられたデータの特徴を学習するのに対し、強化学習は環境から与えられる報酬を目的に行動するエージェントの行動指針 = 方策を学習する。
## 方策関数・価値関数
環境がある状態 $s$ にある時に、エージェントが行動 $a$ を取る確率を出力する関数を方策関数と呼ぶ。またエージェントが方策 $\pi$ に基づいて行動した時、将来得られる価値を出力する関数を価値関数と呼ぶ。価値関数は環境価値関数と行動価値関数がある。環境価値関数は、環境 $s$ の下で方策 $\pi$ を続けた時に将来得られる報酬の期待値を出力する（エージェントが取る行動には依存しない）。$V^{\pi}(s)$ と表記する。行動価値関数は、環境 $s$ の下で行動 $a$ を取り、方策 $\pi$ を続けた場合に将来得られる報酬の期待値を出力する。$Q^{\pi}(a,s)$ と表記する。
## 方策勾配法
方策勾配法は方策 $\pi$ をモデル化して最適化する。方策を決定するパラメータとして $\theta$ を導入し、方策関数を $\pi(a|s,\theta)$ で定義する。また得られる報酬を最大化するための目的関数 $J(\theta)$ を導入し、その勾配を使って $\theta$ を更新する。\
\
$$
\theta \leftarrow \theta + ϵ\nabla J(\theta)
$$\
$J(\theta)$ の勾配は方策勾配定理を用いて次式で計算される。\
\
$$
\nabla J(\theta)=E\left[ \nabla_{\theta} \rm{log} \pi_\theta(a|s)Q^\pi(a,s)\right]
$$


#[Section 2] Alpha Go

##Alpha Go Lee
Policy NetとValue Netの2つのネットワークを学習する。PolicyNetが方策関数、Value Net が行動価値関数であり、両者とも畳み込みNNである。Policy Netは19×19（碁盤のマス目）× 48 ch の入力を持ち、着手予想確率を出力する。Value Net は 19×19×49 の入力を持ち、現局面の勝率を出力する。\
　Alpha Go の学習においては、学習を効率的に進めるために 上記ネットワークに加えてRole Out Policy を用意し、強化学習に先んじてRole out Policy と Polici Net の教師あり学習を行っている。ここで、Role Out policy はPolicy Netよりも簡易的かつ高速な着手予想モデルである。\
　これらの教師あり学習を行ったのち、次にPolicy Netの強化学習を行う。この学習は現状のPolicy Netと過去の学習過程であるPolicy Poolからランダムに選ばれたPolici Netとの対局結果を用いて、方策勾配法により行っている。\
　続いてPolicy Netを用いた対局シミュレーションを行い、その勝敗を教師として Value Net の学習を行ている。囲碁では各局面での盤面の価値を評価することが難しいため、末端の勝敗結果のみを使って探索を行うために、モンテカルロ木探索が用いられている。
##Alpha Go Zero
Alpha Go Zero はAlpha Go Lee と異なり、教師あり学習を行わず強化学習のみで作成している。また入力する特徴を石の配置のみとして、入力データのチャネル数を削減している。さらにPolicy Net と Value Net を一つのネットワークに統合し、途中で枝分かれさえる事でPolicyとValueを出力している。またAlpha Go Zero は2つの畳み込み層を持つResidual Block を重ね合わせた Residual Net を導入している。NNの層が深くなると勾配消失問題が発生するが、Residual Blockは層をスキップするショートカットを持つため、これを回避することができる。 

#[Section 3] 軽量化・高速化技術

## 分散深層学習
現在、機械学習モデルのはコンピューター性能の向上スピードに比べてはるかに速いスピードで複雑さが増している。そこで計算資源（ワーカー）を複数用意して、並列的にニューラルネットを構成しての学習が行われる。

## データ並列化
親モデルを各ワーカーに子モデルとしてコピーし、入力データを分割して各ワーカーに学習させる。学習の進め方は同期型と非同期型がある。同期型は全てのワーカーの学習が終わるのを待ち、各ワーカーの勾配を平均して親モデルの勾配を更新する。非同期型は各ワーカーは自分の学習が終わったら更新した勾配をパラメータサーバーにPushする。次に学習が終了したワーカーはパラメータサーバー上の学習済みモデルを利用して、新たな学習を開始する。

## モデル並列化
親モデルを複数に分割して各ワーカで学習を行う。直列のモデルを分割する場合もあれが、枝分かれした並列構造を分割する場合が多い。

## GPU
CPUが少数の高性能コアで構成されるのに対し、GPUは比較的低性能なコアを多数組み合わせて構成される。簡単な処理を並列化して高速に行う事が可能なので、行列計算を多数行うNNの学習に適する。現在はDeep Learning 用のGPU計算プラットフォームとして NVIDIA が提供する CUDA がよく用いられる。\

## 量子化
パラメータのbit数を小さくすることで、メモリの使用量を削減し、演算を高速化することができる。現在のGPU性能は半精度(16bit)で150 Tera Flops 程度である。計算速度と計算精度はトレードオフの関係があるが、実際の面では高速化のメリットの方が大きい場合が多い。

## 蒸留
精度の高いモデルはニューロンの数が多く、推論時に多くのメモリと演算処理を要する。そこで精度の高い大規模モデルを教師モデルとして、より軽量な生徒モデルを作成する。学習においては教師モデルの重みは固定とし、教師モデルと生徒モデルの両方の誤差を使って生徒モデルの重みを更新する。

## プルーニング
大規模なニュラルネットワークにおいて、予測結果の精度への影響が小さいニューロンを削減することで、モデルを軽量化し計算を高速化することができる。




#[Section 4] 応用モデル

## Mobile Nets
通常の畳み込み演算において、H×W×C の入力データを K×K×C のフィルタで畳み込んでチャネル数 M の出力を得る場合、その計算量は H×W×K×K×C×M となる。Mobile Nets は以下で説明する Depthwise Convolution と Pointwise Convolution を組み合わせる事で計算量を削減する。\
\
Depthwise Convolution\
サイズが K×K×1 のフィルタを用いて入力データのチャネル毎に畳み込みを行う。出力チャネル数は入力チャネル数と同じになる。計算量は H×W×K×K×C となる。\
\
Pointwise Convolution\
サイズが 1×1×M のフィルタを用いて畳み込みを行う。出力チャネルはH×W×M のデータとなる。計算量は H×W×C×M となる。\
\
Mobile Nets は上記2つの畳み込みを組み合わせて出力マップを得るため、計算量は H×W×K×K×C + H×W×C×M となり、通常の畳み込み演算に比べて軽量化が実現できる。

## Dense Net
Dense Netは層が深くなった際の勾配消失問題を解決するために考案されたCNNの一種である。Dense Net は複数の畳み込み層を持つ Dense Block と Dense Block どうしをつなぐ Transition Layer で構成される。Dense Blockでは畳み込み層を通過する事に、前の層の出力を新たなチャネルとして足し合わせる。追加するチャネルの数は growth rate と呼ばれるハイパーパラメータである。Transition Layer では Dense Block で増加したチャネル数の圧縮とプーリング層によるダウンサンプリングが行われる。

## Batch Normalization
与えられた入力データをミニバッチ単位で平均 0、分散 1 となるように変換する。ミニバッチに含まれるデータの各チャネルが一つの分布に従う。バッチサイズは計算機の性能によって変える必要があるため、正規化の効果を計りにくい。またバッチサイズが小さい場合は効果が薄くなってしまう。

## Layer Normalization
与えられた入力データをチャネル単位で正規化する。各データの全てのピクセルが一つの分布に従う。Layer Norm.は入力データのスケーリングや、重み行列のスケーリングおよびシフトに対してロバストである事が知られている。

## Wave Net
与えられた音声データを1次元のつながりを持ったデータとして取り扱い、畳み込みNNによって音声波形を生成する。この時、通常のCNNでは隣り合うデータどうしを畳み込むが、Wave Netでは層が深くなるにつれてデータの間隔を離して畳み込みを行う Dilated convolution が用いられる。この方法によってパラメータに対する受容野を簡単に増やすことができる。


#[Section 5] Transformer

TransformerはSeq2Seqと同様に機械翻訳で良く使われる Encoder-Decoder モデルであるが、RNN を用いずにAttention機構のみを用いる点が Seq2Seq とは異なる。RNNによる逐次処理を行わないので、計算量が非常に少なくなる。

## Encoder
Transformer における Encoder の主要モジュールは Multi-head attention 機構とFeed Foward NN である。ただし、Transformer はRNN が無いため、Embeddingした入力データに対して Position Encoding を行う。Position Encoding は単語の分散ベクトルに sin 関数と cos 関数のパターンを用いて単語の相対的な位置情報を加算する。\
Encoder では入力データに対して Self-Attention を行う。Self-Attention は query, key, value に全て同じテンソルを用いる。またTransformerではSeq2SeqのAttentionのような逐次処理は行わず、全ての時系列データに対するAttention Weight を一括で計算する。\
\
$$
Attention(Q,K,V)=softmax\left( \frac{QK^T}{\sqrt{d_k}}V \right)
$$\
ここで $d_k$ は単語の分散表現ベクトルの次元数である。内積計算で大きすぎる値があると、softmax 関数によってその他の要素が0となり、勾配が消失してしまうため、スケーリングを行って値を小さくしている。\
Transformer のAttention機構はMulti head Attention となっている。入力された query, key, value を複数の組（Head）に分割し、各組についてそれぞれ Attention の計算を行う。一つの大きな Head で計算するよりも、複数の小さな Head で計算を行い、最後に concat する方がパフォーマンスが上がると言われている。\
Attentionからの出力を2層の全結合層である Feed Foward NN に入力する。ここでは単語の位置情報を保持したまま順伝搬を行い、Encodrの出力を得る。

## Decoder
DecoderはEncoderと類似した構成となっているが、Self-Attention 機構の後ろに Source-Target Attention 機構を持ち、Encoder の出力に対する Attention を計算している。また Attention の計算は全ての単語に対して一括で行うため、Decoderにおいては未来の結果を参照しないように Mask 処理を行っている。Attention の出力をFeed Foward NN で変換し、最後に softmax 関数によってトークンの生成確率に変換して、最終的な文章を得る。 


#[Section 6] 物体検知・セグメンテーション

画像認識のタスクは主に下記の4つに分類される。\
・分類問題 (Classification)\
・物体検知 (Object detection)\
　　物体の位置を Bounding Box で囲む。\
・意味領域分割 (semantic segmentation)\
　　各ピクセルにクラスラベルを割り当てる。\
・個体領域分割 (Instance segmentation)\
　　各ピクセルにクラスラベルを割り当て、さらに各インスタンスを区別する。





# データセット
　画像認識の学習や精度評価においては、目的に応じて適切なデータセットを選択する必要がある。クラス数、データ数（学習用＋評価用）に加えて、一枚の画像あたりに対象が含まれる数 (Box/画像) も考慮に入れるのが良い。Box/画像 が小さいとアイコン的な映りとなる。大きいと対象が小さくなったり部分的な重なりが発生する傾向がある。





# 物体検出における評価指標
物体検出においては、クラスラベルの正誤だけではなく、物体位置の予測精度も評価したいので、評価指標にIoU (Intersection of Union) を用いる。\
\
$$
\mbox{IoU}=\frac{\mbox{Area of Overlap}}{\mbox{Area of Union}}
$$\
ここで、以下は評価指標として誤りであることに注意する。\
\
$$
\mbox{IoU}=\frac{\mbox{Area of Overlap}}{\mbox{Area of Grand Truth BB}}, \quad
\mbox{IoU}=\frac{\mbox{Area of Overlap}}{\mbox{Area of Prediction}}
$$\
物体検知における Presision 、Recall の評価は、confidence（以下 conf.）とIOU のそれぞれの閾値に対して、$\rm conf.>th_{conf}$ かつ $\rm IoU>th_{IoU}$ の時 TP (True Positive) となる。ここで、検出対象に対してより高い conf. で既に TP と判定されている場合は、条件を満たしている場合でも FP となる事に注意する。
ある一つのクラスラベルに対して、$\rm th_{IoU}$ を固定し、$\rm th_{conf.}$ を0～1 の範囲で変化させると、Recall に対する Presision の変化を表す PR 曲線が描ける。この曲線を Recall = 0～1 の範囲で積分した値を Average Presision : AP と呼ぶ。また全ての分類クラスの AP を平均したものを maan Average Presision : mAP と呼ぶ。1\
\
$$
\rm{AP} = \int_{0}^{1}P(R)dR\\
\rm{mAP} = \frac{1}{C}\sum_{i=1}^CAP_i
$$\
物体検出のフレームワークは様々なものがあるが、大きくは1段階の検出を行うものと2段階の検出を行うもので分類される。1段階検出器は物体位置の検出と分類を一つのネットワークで行うが、2段階検出器は最初に位置を検出し、そこで抽出した画像に対して分類を行う。相対的に1段階検出器は検出速度に勝り、2段階検出器は検出精度に勝る。












# SSD (Single Shot Detector)
SSDは1段階検出器に分類される物体検出用フレームワークであり、VGG16 CNN を基本構造としている。SSDはサイズの異なる複数の畳み込み層から特徴マップを出力する。各特徴マップは一つの特徴量毎にBounding Box (Default Box) を持ち、一つのDefault Boxは クラス数+4 の結果を出力する（4 はBBの位置を表す指標）。特徴マップは層が深くなるほど解像度が低くなるので、浅い層のBBは比較的小さな物体を検出し、深い層のBBは大きな物体を検出する。





# Semantic Segmentation
Semantic Secmentation においては、入力画像の各ピクセル毎に分類を行うため、畳み込みによって得られた特徴マップを入力画像と同じ解像度に Upsampling する必要がある。この工程を Deconvolution と呼ぶ。ここで注意すべき点として、Pooling 処理によって失われたローカルな情報が Deconvolution によって復元するわけではない。そこで低レイヤーの畳み込み層で出力された特徴マップを Upsampling した特徴マップに要素ごとに足し合わせて、失われた輪郭情報を補完することが行われる。


# 実装演習 Transformer

In [ ]:
def position_encoding_init(n_position, d_pos_vec):
    """
    Positional Encodingのための行列の初期化を行う
    :param n_position: int, 系列長
    :param d_pos_vec: int, 隠れ層の次元数
    :return torch.tensor, size=(n_position, d_pos_vec)
    """
    # PADがある単語の位置はpos=0にしておき、position_encも0にする
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / d_pos_vec) for j in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return torch.tensor(position_enc, dtype=torch.float)

In [ ]:
class ScaledDotProductAttention(nn.Module):
    
    def __init__(self, d_model, attn_dropout=0.1):
        """
        :param d_model: int, 隠れ層の次元数
        :param attn_dropout: float, ドロップアウト率
        """
        super(ScaledDotProductAttention, self).__init__()
        self.temper = np.power(d_model, 0.5)  # スケーリング因子
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, attn_mask):
        """
        :param q: torch.tensor, queryベクトル, 
            size=(n_head*batch_size, len_q, d_model/n_head)
        :param k: torch.tensor, key, 
            size=(n_head*batch_size, len_k, d_model/n_head)
        :param v: torch.tensor, valueベクトル, 
            size=(n_head*batch_size, len_v, d_model/n_head)
        :param attn_mask: torch.tensor, Attentionに適用するマスク, 
            size=(n_head*batch_size, len_q, len_k)
        :return output: 出力ベクトル, 
            size=(n_head*batch_size, len_q, d_model/n_head)
        :return attn: Attention
            size=(n_head*batch_size, len_q, len_k)
        """
        # QとKの内積でAttentionの重みを求め、スケーリングする
        attn = torch.bmm(q, k.transpose(1, 2)) / self.temper  # (n_head*batch_size, len_q, len_k)
        # Attentionをかけたくない部分がある場合は、その部分を負の無限大に飛ばしてSoftmaxの値が0になるようにする
        attn.data.masked_fill_(attn_mask, -float('inf'))
        
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v)

        return output, attn

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        """
        :param n_head: int, ヘッド数
        :param d_model: int, 隠れ層の次元数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param dropout: float, ドロップアウト率
        """
        super(MultiHeadAttention, self).__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        # 各ヘッドごとに異なる重みで線形変換を行うための重み
        # nn.Parameterを使うことで、Moduleのパラメータとして登録できる. TFでは更新が必要な変数はtf.Variableでラップするのでわかりやすい
        self.w_qs = nn.Parameter(torch.empty([n_head, d_model, d_k], dtype=torch.float))
        self.w_ks = nn.Parameter(torch.empty([n_head, d_model, d_k], dtype=torch.float))
        self.w_vs = nn.Parameter(torch.empty([n_head, d_model, d_v], dtype=torch.float))
        # nn.init.xavier_normal_で重みの値を初期化
        nn.init.xavier_normal_(self.w_qs)
        nn.init.xavier_normal_(self.w_ks)
        nn.init.xavier_normal_(self.w_vs)

        self.attention = ScaledDotProductAttention(d_model)
        self.layer_norm = nn.LayerNorm(d_model) # 各層においてバイアスを除く活性化関数への入力を平均０、分散１に正則化
        self.proj = nn.Linear(n_head*d_v, d_model)  # 複数ヘッド分のAttentionの結果を元のサイズに写像するための線形層
        # nn.init.xavier_normal_で重みの値を初期化
        nn.init.xavier_normal_(self.proj.weight)
        
        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, attn_mask=None):
        """
        :param q: torch.tensor, queryベクトル, 
            size=(batch_size, len_q, d_model)
        :param k: torch.tensor, key, 
            size=(batch_size, len_k, d_model)
        :param v: torch.tensor, valueベクトル, 
            size=(batch_size, len_v, d_model)
        :param attn_mask: torch.tensor, Attentionに適用するマスク, 
            size=(batch_size, len_q, len_k)
        :return outputs: 出力ベクトル, 
            size=(batch_size, len_q, d_model)
        :return attns: Attention
            size=(n_head*batch_size, len_q, len_k)
            
        """
        d_k, d_v = self.d_k, self.d_v
        n_head = self.n_head

        # residual connectionのための入力 出力に入力をそのまま加算する
        residual = q

        batch_size, len_q, d_model = q.size()
        batch_size, len_k, d_model = k.size()
        batch_size, len_v, d_model = v.size()

        # 複数ヘッド化
        # torch.repeat または .repeatで指定したdimに沿って同じテンソルを作成
        q_s = q.repeat(n_head, 1, 1) # (n_head*batch_size, len_q, d_model)
        k_s = k.repeat(n_head, 1, 1) # (n_head*batch_size, len_k, d_model)
        v_s = v.repeat(n_head, 1, 1) # (n_head*batch_size, len_v, d_model)
        # ヘッドごとに並列計算させるために、n_headをdim=0に、batch_sizeをdim=1に寄せる
        q_s = q_s.view(n_head, -1, d_model) # (n_head, batch_size*len_q, d_model)
        k_s = k_s.view(n_head, -1, d_model) # (n_head, batch_size*len_k, d_model)
        v_s = v_s.view(n_head, -1, d_model) # (n_head, batch_size*len_v, d_model)

        # 各ヘッドで線形変換を並列計算(p16左側`Linear`)
        q_s = torch.bmm(q_s, self.w_qs)  # (n_head, batch_size*len_q, d_k)
        k_s = torch.bmm(k_s, self.w_ks)  # (n_head, batch_size*len_k, d_k)
        v_s = torch.bmm(v_s, self.w_vs)  # (n_head, batch_size*len_v, d_v)
        # Attentionは各バッチ各ヘッドごとに計算させるためにbatch_sizeをdim=0に寄せる
        q_s = q_s.view(-1, len_q, d_k)   # (n_head*batch_size, len_q, d_k)
        k_s = k_s.view(-1, len_k, d_k)   # (n_head*batch_size, len_k, d_k)
        v_s = v_s.view(-1, len_v, d_v)   # (n_head*batch_size, len_v, d_v)

        # Attentionを計算(p16.左側`Scaled Dot-Product Attention * h`)
        outputs, attns = self.attention(q_s, k_s, v_s, attn_mask=attn_mask.repeat(n_head, 1, 1))

        # 各ヘッドの結果を連結(p16左側`Concat`)
        # torch.splitでbatch_sizeごとのn_head個のテンソルに分割
        outputs = torch.split(outputs, batch_size, dim=0)  # (batch_size, len_q, d_model) * n_head
        # dim=-1で連結
        outputs = torch.cat(outputs, dim=-1)  # (batch_size, len_q, d_model*n_head)

        # residual connectionのために元の大きさに写像(p16左側`Linear`)
        outputs = self.proj(outputs)  # (batch_size, len_q, d_model)
        outputs = self.dropout(outputs)
        outputs = self.layer_norm(outputs + residual)

        return outputs, attns

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """
    :param d_hid: int, 隠れ層1層目の次元数
    :param d_inner_hid: int, 隠れ層2層目の次元数
    :param dropout: float, ドロップアウト率
    """
    def __init__(self, d_hid, d_inner_hid, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # window size 1のconv層を定義することでPosition wiseな全結合層を実現する.
        self.w_1 = nn.Conv1d(d_hid, d_inner_hid, 1)
        self.w_2 = nn.Conv1d(d_inner_hid, d_hid, 1)
        self.layer_norm = nn.LayerNorm(d_hid)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        :param x: torch.tensor,
            size=(batch_size, max_length, d_hid)
        :return: torch.tensor,
            size=(batch_size, max_length, d_hid) 
        """
        residual = x
        output = self.relu(self.w_1(x.transpose(1, 2)))
        output = self.w_2(output).transpose(2, 1)
        output = self.dropout(output)
        return self.layer_norm(output + residual)

In [ ]:
def get_attn_padding_mask(seq_q, seq_k):
    """
    keyのPADに対するattentionを0にするためのマスクを作成する
    :param seq_q: tensor, queryの系列, size=(batch_size, len_q)
    :param seq_k: tensor, keyの系列, size=(batch_size, len_k)
    :return pad_attn_mask: tensor, size=(batch_size, len_q, len_k)
    """
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(PAD).unsqueeze(1)   # (N, 1, len_k) PAD以外のidを全て0にする
    pad_attn_mask = pad_attn_mask.expand(batch_size, len_q, len_k) # (N, len_q, len_k)
    return pad_attn_mask

In [ ]:
def get_attn_subsequent_mask(seq):
    """
    未来の情報に対するattentionを0にするためのマスクを作成する
    :param seq: tensor, size=(batch_size, length)
    :return subsequent_mask: tensor, size=(batch_size, length, length)
    """
    attn_shape = (seq.size(1), seq.size(1))
    # 上三角行列(diagonal=1: 対角線より上が1で下が0)
    subsequent_mask = torch.triu(torch.ones(attn_shape, dtype=torch.uint8, device=device), diagonal=1)
    subsequent_mask = subsequent_mask.repeat(seq.size(0), 1, 1)
    return subsequent_mask

In [ ]:
class EncoderLayer(nn.Module):
    """Encoderのブロックのクラス"""
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
        """
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param dropout: float, ドロップアウト率
        """
        super(EncoderLayer, self).__init__()
        # Encoder内のSelf-Attention
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)
        # Postionwise FFN
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)

    def forward(self, enc_input, slf_attn_mask=None):
        """
        :param enc_input: tensor, Encoderの入力, 
            size=(batch_size, max_length, d_model)
        :param slf_attn_mask: tensor, Self Attentionの行列にかけるマスク, 
            size=(batch_size, len_q, len_k)
        :return enc_output: tensor, Encoderの出力, 
            size=(batch_size, max_length, d_model)
        :return enc_slf_attn: tensor, EncoderのSelf Attentionの行列, 
            size=(n_head*batch_size, len_q, len_k)
        """
        # Self-Attentionのquery, key, valueにはすべてEncoderの入力（enc_input）が入る
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, attn_mask=slf_attn_mask)
        enc_output = self.pos_ffn(enc_output)
        return enc_output, enc_slf_attn

In [ ]:
class Encoder(nn.Module):
    """EncoderLayerブロックからなるEncoderのクラス"""
    def __init__(
            self, n_src_vocab, max_length, n_layers=6, n_head=8, d_k=64, d_v=64,
            d_word_vec=512, d_model=512, d_inner_hid=1024, dropout=0.1):
        """
        :param n_src_vocab: int, 入力言語の語彙数
        :param max_length: int, 最大系列長
        :param n_layers: int, レイヤー数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param d_word_vec: int, 単語の埋め込みの次元数
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param dropout: float, ドロップアウト率        
        """
        super(Encoder, self).__init__()

        n_position = max_length + 1
        self.max_length = max_length
        self.d_model = d_model

        # Positional Encodingを用いたEmbedding
        self.position_enc = nn.Embedding(n_position, d_word_vec, padding_idx=PAD)
        self.position_enc.weight.data = position_encoding_init(n_position, d_word_vec)

        # 一般的なEmbedding
        self.src_word_emb = nn.Embedding(n_src_vocab, d_word_vec, padding_idx=PAD)

        # EncoderLayerをn_layers個積み重ねる
        self.layer_stack = nn.ModuleList([
            EncoderLayer(d_model, d_inner_hid, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])

    def forward(self, src_seq, src_pos):
        """
        :param src_seq: tensor, 入力系列, 
            size=(batch_size, max_length)
        :param src_pos: tensor, 入力系列の各単語の位置情報,
            size=(batch_size, max_length)
        :return enc_output: tensor, Encoderの最終出力, 
            size=(batch_size, max_length, d_model)
        :return enc_slf_attns: list, EncoderのSelf Attentionの行列のリスト
        """
        # 一般的な単語のEmbeddingを行う
        enc_input = self.src_word_emb(src_seq)
        # Positional EncodingのEmbeddingを加算する
        enc_input += self.position_enc(src_pos)

        enc_slf_attns = []
        enc_output = enc_input
        # key(=enc_input)のPADに対応する部分のみ1のマスクを作成
        enc_slf_attn_mask = get_attn_padding_mask(src_seq, src_seq)

        # n_layers個のEncoderLayerに入力を通す
        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(
                enc_output, slf_attn_mask=enc_slf_attn_mask)
            enc_slf_attns += [enc_slf_attn]

        return enc_output, enc_slf_attns

In [ ]:
class DecoderLayer(nn.Module):
    """Decoderのブロックのクラス"""
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
        """
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param dropout: float, ドロップアウト率
        """
        super(DecoderLayer, self).__init__()
        # Decoder内のSelf-Attention
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        # Encoder-Decoder間のSource-Target Attention
        self.enc_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        # Positionwise FFN
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)

    def forward(self, dec_input, enc_output, slf_attn_mask=None, dec_enc_attn_mask=None):
        """
        :param dec_input: tensor, Decoderの入力, 
            size=(batch_size, max_length, d_model)
        :param enc_output: tensor, Encoderの出力, 
            size=(batch_size, max_length, d_model)
        :param slf_attn_mask: tensor, Self Attentionの行列にかけるマスク, 
            size=(batch_size, len_q, len_k)
        :param dec_enc_attn_mask: tensor, Soutce-Target Attentionの行列にかけるマスク, 
            size=(batch_size, len_q, len_k)
        :return dec_output: tensor, Decoderの出力, 
            size=(batch_size, max_length, d_model)
        :return dec_slf_attn: tensor, DecoderのSelf Attentionの行列, 
            size=(n_head*batch_size, len_q, len_k)
        :return dec_enc_attn: tensor, DecoderのSoutce-Target Attentionの行列, 
            size=(n_head*batch_size, len_q, len_k)
        """
        # Self-Attentionのquery, key, valueにはすべてDecoderの入力（dec_input）が入る
        dec_output, dec_slf_attn = self.slf_attn(
            dec_input, dec_input, dec_input, attn_mask=slf_attn_mask)
        # Source-Target-AttentionのqueryにはDecoderの出力(dec_output), key, valueにはEncoderの出力（enc_output）が入る
        dec_output, dec_enc_attn = self.enc_attn(
            dec_output, enc_output, enc_output, attn_mask=dec_enc_attn_mask)
        dec_output = self.pos_ffn(dec_output)

        return dec_output, dec_slf_attn, dec_enc_attn

In [ ]:
class Decoder(nn.Module):
    """DecoderLayerブロックからなるDecoderのクラス"""
    def __init__(
            self, n_tgt_vocab, max_length, n_layers=6, n_head=8, d_k=64, d_v=64,
            d_word_vec=512, d_model=512, d_inner_hid=1024, dropout=0.1):
        """
        :param n_tgt_vocab: int, 出力言語の語彙数
        :param max_length: int, 最大系列長
        :param n_layers: int, レイヤー数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param d_word_vec: int, 単語の埋め込みの次元数
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param dropout: float, ドロップアウト率        
        """
        super(Decoder, self).__init__()
        n_position = max_length + 1
        self.max_length = max_length
        self.d_model = d_model

        # Positional Encodingを用いたEmbedding
        self.position_enc = nn.Embedding(
            n_position, d_word_vec, padding_idx=PAD)
        self.position_enc.weight.data = position_encoding_init(n_position, d_word_vec)

        # 一般的なEmbedding
        self.tgt_word_emb = nn.Embedding(
            n_tgt_vocab, d_word_vec, padding_idx=PAD)
        self.dropout = nn.Dropout(dropout)

        # DecoderLayerをn_layers個積み重ねる
        self.layer_stack = nn.ModuleList([
            DecoderLayer(d_model, d_inner_hid, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])

    def forward(self, tgt_seq, tgt_pos, src_seq, enc_output):
        """
        :param tgt_seq: tensor, 出力系列, 
            size=(batch_size, max_length)
        :param tgt_pos: tensor, 出力系列の各単語の位置情報,
            size=(batch_size, max_length)
        :param src_seq: tensor, 入力系列, 
            size=(batch_size, n_src_vocab)
        :param enc_output: tensor, Encoderの出力, 
            size=(batch_size, max_length, d_model)
        :return dec_output: tensor, Decoderの最終出力, 
            size=(batch_size, max_length, d_model)
        :return dec_slf_attns: list, DecoderのSelf Attentionの行列のリスト 
        :return dec_slf_attns: list, DecoderのSelf Attentionの行列のリスト
        """
        # 一般的な単語のEmbeddingを行う
        dec_input = self.tgt_word_emb(tgt_seq)
        # Positional EncodingのEmbeddingを加算する
        dec_input += self.position_enc(tgt_pos)

        # Self-Attention用のマスクを作成
        # key(=dec_input)のPADに対応する部分が1のマスクと、queryから見たkeyの未来の情報に対応する部分が1のマスクのORをとる
        dec_slf_attn_pad_mask = get_attn_padding_mask(tgt_seq, tgt_seq)  # (N, max_length, max_length)
        dec_slf_attn_sub_mask = get_attn_subsequent_mask(tgt_seq)  # (N, max_length, max_length)
        dec_slf_attn_mask = torch.gt(dec_slf_attn_pad_mask + dec_slf_attn_sub_mask, 0)  # ORをとる

        # key(=dec_input)のPADに対応する部分のみ1のマスクを作成
        dec_enc_attn_pad_mask = get_attn_padding_mask(tgt_seq, src_seq)  # (N, max_length, max_length)

        dec_slf_attns, dec_enc_attns = [], []

        dec_output = dec_input
        # n_layers個のDecoderLayerに入力を通す
        for dec_layer in self.layer_stack:
            dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                dec_output, enc_output,
                slf_attn_mask=dec_slf_attn_mask,
                dec_enc_attn_mask=dec_enc_attn_pad_mask)

            dec_slf_attns += [dec_slf_attn]
            dec_enc_attns += [dec_enc_attn]

        return dec_output, dec_slf_attns, dec_enc_attns

In [ ]:
class Transformer(nn.Module):
    """Transformerのモデル全体のクラス"""
    def __init__(
            self, n_src_vocab, n_tgt_vocab, max_length, n_layers=6, n_head=8,
            d_word_vec=512, d_model=512, d_inner_hid=1024, d_k=64, d_v=64,
            dropout=0.1, proj_share_weight=True):
        """
        :param n_src_vocab: int, 入力言語の語彙数
        :param n_tgt_vocab: int, 出力言語の語彙数
        :param max_length: int, 最大系列長
        :param n_layers: int, レイヤー数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param d_word_vec: int, 単語の埋め込みの次元数
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param dropout: float, ドロップアウト率        
        :param proj_share_weight: bool, 出力言語の単語のEmbeddingと出力の写像で重みを共有する        
        """
        super(Transformer, self).__init__()
        self.encoder = Encoder(
            n_src_vocab, max_length, n_layers=n_layers, n_head=n_head,
            d_word_vec=d_word_vec, d_model=d_model,
            d_inner_hid=d_inner_hid, dropout=dropout)
        self.decoder = Decoder(
            n_tgt_vocab, max_length, n_layers=n_layers, n_head=n_head,
            d_word_vec=d_word_vec, d_model=d_model,
            d_inner_hid=d_inner_hid, dropout=dropout)
        self.tgt_word_proj = nn.Linear(d_model, n_tgt_vocab, bias=False)
        nn.init.xavier_normal_(self.tgt_word_proj.weight)
        self.dropout = nn.Dropout(dropout)

        assert d_model == d_word_vec  # 各モジュールの出力のサイズは揃える

        if proj_share_weight:
            # 出力言語の単語のEmbeddingと出力の写像で重みを共有する
            assert d_model == d_word_vec
            self.tgt_word_proj.weight = self.decoder.tgt_word_emb.weight

    def get_trainable_parameters(self):
        # Positional Encoding以外のパラメータを更新する
        enc_freezed_param_ids = set(map(id, self.encoder.position_enc.parameters()))
        dec_freezed_param_ids = set(map(id, self.decoder.position_enc.parameters()))
        freezed_param_ids = enc_freezed_param_ids | dec_freezed_param_ids
        return (p for p in self.parameters() if id(p) not in freezed_param_ids)

    def forward(self, src, tgt):
        src_seq, src_pos = src
        tgt_seq, tgt_pos = tgt

        src_seq = src_seq[:, 1:]
        src_pos = src_pos[:, 1:]
        tgt_seq = tgt_seq[:, :-1]
        tgt_pos = tgt_pos[:, :-1]

        enc_output, *_ = self.encoder(src_seq, src_pos)
        dec_output, *_ = self.decoder(tgt_seq, tgt_pos, src_seq, enc_output)
        seq_logit = self.tgt_word_proj(dec_output)

        return seq_logit

In [ ]:
def compute_loss(batch_X, batch_Y, model, criterion, optimizer=None, is_train=True):
    # バッチの損失を計算
    model.train(is_train)
    
    pred_Y = model(batch_X, batch_Y)
    gold = batch_Y[0][:, 1:].contiguous()
#     gold = batch_Y[0].contiguous()
    loss = criterion(pred_Y.view(-1, pred_Y.size(2)), gold.view(-1))

    if is_train:  # 訓練時はパラメータを更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    gold = gold.data.cpu().numpy().tolist()
    pred = pred_Y.max(dim=-1)[1].data.cpu().numpy().tolist()

    return loss.item(), gold, pred

In [ ]:
MAX_LENGTH = 20
batch_size = 64
num_epochs = 15
lr = 0.001
ckpt_path = 'transformer.pth'
max_length = MAX_LENGTH + 2

In [ ]:
model_args = {
    'n_src_vocab': vocab_size_X,
    'n_tgt_vocab': vocab_size_Y,
    'max_length': max_length,
    'proj_share_weight': True,
    'd_k': 32,
    'd_v': 32,
    'd_model': 128,
    'd_word_vec': 128,
    'd_inner_hid': 256,
    'n_layers': 3,
    'n_head': 6,
    'dropout': 0.1,
}

In [ ]:
# DataLoaderやモデルを定義
train_dataloader = DataLoader(
    train_X, train_Y, batch_size
    )
valid_dataloader = DataLoader(
    valid_X, valid_Y, batch_size, 
    shuffle=False
    )

model = Transformer(**model_args).to(device)

optimizer = optim.Adam(model.get_trainable_parameters(), lr=lr)

criterion = nn.CrossEntropyLoss(ignore_index=PAD, size_average=False).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
def calc_bleu(refs, hyps):
    """
    BLEUスコアを計算する関数
    :param refs: list, 参照訳。単語のリストのリスト (例： [['I', 'have', 'a', 'pen'], ...])
    :param hyps: list, モデルの生成した訳。単語のリストのリスト (例： [['I', 'have', 'a', 'pen'], ...])
    :return: float, BLEUスコア(0~100)
    """
    refs = [[ref[:ref.index(EOS)]] for ref in refs]
    hyps = [hyp[:hyp.index(EOS)] if EOS in hyp else hyp for hyp in hyps]
    return 100 * bleu_score.corpus_bleu(refs, hyps)

In [ ]:
# 訓練
best_valid_bleu = 0.

for epoch in range(1, num_epochs+1):
    start = time.time()
    train_loss = 0.
    train_refs = []
    train_hyps = []
    valid_loss = 0.
    valid_refs = []
    valid_hyps = []
    # train
    for batch in train_dataloader:
        batch_X, batch_Y = batch
        loss, gold, pred = compute_loss(
            batch_X, batch_Y, model, criterion, optimizer, is_train=True
            )
        train_loss += loss
        train_refs += gold
        train_hyps += pred
    # valid
    for batch in valid_dataloader:
        batch_X, batch_Y = batch
        loss, gold, pred = compute_loss(
            batch_X, batch_Y, model, criterion, is_train=False
            )
        valid_loss += loss
        valid_refs += gold
        valid_hyps += pred
    # 損失をサンプル数で割って正規化
    train_loss /= len(train_dataloader.data) 
    valid_loss /= len(valid_dataloader.data) 
    # BLEUを計算
    train_bleu = calc_bleu(train_refs, train_hyps)
    valid_bleu = calc_bleu(valid_refs, valid_hyps)

    # validationデータでBLEUが改善した場合にはモデルを保存
    if valid_bleu > best_valid_bleu:
        ckpt = model.state_dict()
        torch.save(ckpt, ckpt_path)
        best_valid_bleu = valid_bleu

    elapsed_time = (time.time()-start) / 60
    print('Epoch {} [{:.1f}min]: train_loss: {:5.2f}  train_bleu: {:2.2f}  valid_loss: {:5.2f}  valid_bleu: {:2.2f}'.format(
            epoch, elapsed_time, train_loss, train_bleu, valid_loss, valid_bleu))
    print('-'*80)

Epoch 1 [0.6min]: train_loss: 77.34  train_bleu: 4.77  valid_loss: 41.31  valid_bleu: 11.27
--------------------------------------------------------------------------------
Epoch 2 [0.6min]: train_loss: 39.27  train_bleu: 12.31  valid_loss: 32.12  valid_bleu: 17.52
--------------------------------------------------------------------------------
Epoch 3 [0.6min]: train_loss: 31.83  train_bleu: 18.21  valid_loss: 28.06  valid_bleu: 21.84
--------------------------------------------------------------------------------
Epoch 4 [0.6min]: train_loss: 28.11  train_bleu: 21.84  valid_loss: 25.58  valid_bleu: 25.02
--------------------------------------------------------------------------------
Epoch 5 [0.6min]: train_loss: 25.67  train_bleu: 24.76  valid_loss: 24.24  valid_bleu: 27.06
--------------------------------------------------------------------------------
Epoch 6 [0.7min]: train_loss: 23.86  train_bleu: 26.99  valid_loss: 22.98  valid_bleu: 29.12
--------------------------------------